In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
from scipy.spatial.distance import cosine

In [3]:

# Get our models - The package will take care of downloading the models automatically
# For best performance: Muennighoff/SGPT-5.8B-weightedmean-nli-bitfit
# Muennighoff/SGPT-125M-weightedmean-nli-bitfit
tokenizer = AutoTokenizer.from_pretrained("Muennighoff/SGPT-1.3B-weightedmean-nli-bitfit")
model = AutoModel.from_pretrained("Muennighoff/SGPT-1.3B-weightedmean-nli-bitfit")

# Tokenize input texts
texts = [
    "Very good person",
    "Humanitarian and kind",
    "Very bad person",
    "He hurts other people",
]
batch_tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    # Get hidden state of shape [bs, seq_len, hid_dim]
    last_hidden_state = model(**batch_tokens, output_hidden_states=True, return_dict=True).last_hidden_state

# Get weights of shape [bs, seq_len, hid_dim]
weights = (
    torch.arange(start=1, end=last_hidden_state.shape[1] + 1)
    .unsqueeze(0)
    .unsqueeze(-1)
    .expand(last_hidden_state.size())
    .float().to(last_hidden_state.device)
)

# Get attn mask of shape [bs, seq_len, hid_dim]
input_mask_expanded = (
    batch_tokens["attention_mask"]
    .unsqueeze(-1)
    .expand(last_hidden_state.size())
    .float()
)

# Perform weighted mean pooling across seq_len: bs, seq_len, hidden_dim -> bs, hidden_dim
sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded * weights, dim=1)
sum_mask = torch.sum(input_mask_expanded * weights, dim=1)

embeddings = sum_embeddings / sum_mask
print(embeddings.norm(p=2, dim=1)[:, None].shape)
embeddings = embeddings / embeddings.norm(p=2, dim=1)[:, None]
rdm = embeddings @ embeddings.T

# Calculate cosine similarities
# Cosine similarities are in [-1, 1]. Higher means more similar
cosine_sim_0_1 = 1 - cosine(embeddings[0], embeddings[1])
cosine_sim_0_2 = 1 - cosine(embeddings[0], embeddings[2])
cosine_sim_0_3 = 1 - cosine(embeddings[0], embeddings[3])

print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[1], cosine_sim_0_1))
print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[2], cosine_sim_0_2))
print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[3], cosine_sim_0_3))

torch.Size([4, 1])
Cosine similarity between "Very good person" and "Humanitarian and kind" is: 0.429
Cosine similarity between "Very good person" and "Very bad person" is: 0.091
Cosine similarity between "Very good person" and "He hurts other people" is: 0.021


In [6]:
import plotly.express as px
import pandas as pd

In [8]:
rdm = pd.DataFrame(rdm, index=texts, columns=texts)

In [9]:
px.imshow(rdm)

In [19]:
from sklearn.manifold import TSNE

x_embedded = TSNE(n_components=3, metric='precomputed', learning_rate='auto').fit_transform(rdm)
print(x_embedded)
x_embedded = pd.DataFrame(x_embedded, index=texts, columns=['x', 'y', 'z'])
print(x_embedded)
px.scatter_3d(x=x_embedded['x'], y=x_embedded['y'], z=x_embedded['z'], text = x_embedded.index)

[[-383.3057     199.75693   -430.5174   ]
 [  -6.1293507  214.89766   -170.54361  ]
 [ 315.70306   -263.8649     556.1793   ]
 [  90.52992    -96.87527    183.30766  ]]
                                x           y           z
Very good person      -383.305695  199.756927 -430.517395
Humanitarian and kind   -6.129351  214.897659 -170.543610
Very bad person        315.703064 -263.864899  556.179321
He hurts other people   90.529922  -96.875267  183.307663


/home/administrator/anaconda3/envs/clip_decoder/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/administrator/anaconda3/envs/clip_decoder/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:819: FutureWarning:

'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.

